In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Soccer player data').getOrCreate()
spark

In [5]:
path = 'Datasets/'
players_data = spark.read.csv(path + 'player.csv',inferSchema=True,header=True)
players_data.show(5)

+---+-------------+------------------+------------------+-------------------+------+------+
| id|player_api_id|       player_name|player_fifa_api_id|           birthday|height|weight|
+---+-------------+------------------+------------------+-------------------+------+------+
|  1|       505942|Aaron Appindangoye|            218353|1992-02-29 00:00:00|182.88|   187|
|  2|       155782|   Aaron Cresswell|            189615|1989-12-15 00:00:00|170.18|   146|
|  3|       162549|       Aaron Doran|            186170|1991-05-13 00:00:00|170.18|   163|
|  4|        30572|     Aaron Galindo|            140161|1982-05-08 00:00:00|182.88|   198|
|  5|        23780|      Aaron Hughes|             17725|1979-11-08 00:00:00|182.88|   154|
+---+-------------+------------------+------------------+-------------------+------+------+
only showing top 5 rows



In [8]:
players_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- player_api_id: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_fifa_api_id: integer (nullable = true)
 |-- birthday: string (nullable = true)
 |-- height: double (nullable = true)
 |-- weight: integer (nullable = true)



In [9]:
players_data.count()

11060

In [10]:
len(players_data.columns)

7

In [12]:
player_attributes = spark.read.csv(path + 'player_attributes.csv',inferSchema=True,header=True)
player_attributes.limit(5).toPandas()

c:\users\aditya\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
1,2,218353,505942,2015-11-19 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
2,3,218353,505942,2015-09-21 00:00:00,62,66,right,medium,medium,49,...,54,48,65,66,69,6,11,10,8,8
3,4,218353,505942,2015-03-20 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7
4,5,218353,505942,2007-02-22 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7


In [15]:
player_attributes.select('player_api_id').distinct().count()

11060

In [16]:
player_attributes.count()

183978

In [21]:
players_data = players_data.drop('id','player_fifa_api_id')

In [22]:
player_attributes.printSchema()

root
 |-- id: integer (nullable = true)
 |-- player_fifa_api_id: integer (nullable = true)
 |-- player_api_id: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- overall_rating: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- preferred_foot: string (nullable = true)
 |-- attacking_work_rate: string (nullable = true)
 |-- defensive_work_rate: string (nullable = true)
 |-- crossing: integer (nullable = true)
 |-- finishing: integer (nullable = true)
 |-- heading_accuracy: integer (nullable = true)
 |-- short_passing: integer (nullable = true)
 |-- volleys: integer (nullable = true)
 |-- dribbling: integer (nullable = true)
 |-- curve: integer (nullable = true)
 |-- free_kick_accuracy: integer (nullable = true)
 |-- long_passing: integer (nullable = true)
 |-- ball_control: integer (nullable = true)
 |-- acceleration: integer (nullable = true)
 |-- sprint_speed: integer (nullable = true)
 |-- agility: integer (nullable = true)
 |-- reactions: integ

In [24]:
player_attributes = player_attributes.drop('id','player_fifa_api_id','preferred_foot','attacking_work_rate','defensive_work_rate','crossing','jumping'
                      ,'sprint_speed','balance','aggression','short_passing','potential')

In [25]:
player_attributes = player_attributes.dropna()
players_data = players_data.dropna()

In [36]:
from pyspark.sql.functions import udf,round

In [27]:
year_extract_udf = udf(lambda data: data.split('-')[0])

player_attributes = player_attributes.withColumn('year',
                                                year_extract_udf(player_attributes.date))

In [28]:
player_attributes.printSchema()

root
 |-- player_api_id: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- overall_rating: integer (nullable = true)
 |-- finishing: integer (nullable = true)
 |-- heading_accuracy: integer (nullable = true)
 |-- volleys: integer (nullable = true)
 |-- dribbling: integer (nullable = true)
 |-- curve: integer (nullable = true)
 |-- free_kick_accuracy: integer (nullable = true)
 |-- long_passing: integer (nullable = true)
 |-- ball_control: integer (nullable = true)
 |-- acceleration: integer (nullable = true)
 |-- agility: integer (nullable = true)
 |-- reactions: integer (nullable = true)
 |-- shot_power: integer (nullable = true)
 |-- stamina: integer (nullable = true)
 |-- strength: integer (nullable = true)
 |-- long_shots: integer (nullable = true)
 |-- interceptions: integer (nullable = true)
 |-- positioning: integer (nullable = true)
 |-- vision: integer (nullable = true)
 |-- penalties: integer (nullable = true)
 |-- marking: integer (nullable = true)
 |-- stan

In [29]:
player_attributes.limit(5).toPandas()

,player_api_id,date,overall_rating,finishing,heading_accuracy,volleys,dribbling,curve,free_kick_accuracy,long_passing,...,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,year
0,505942,2016-02-18 00:00:00,67,44,71,44,51,45,39,64,...,48,65,69,69,6,11,10,8,8,2016
1,505942,2015-11-19 00:00:00,67,44,71,44,51,45,39,64,...,48,65,69,69,6,11,10,8,8,2015
2,505942,2015-09-21 00:00:00,62,44,71,44,51,45,39,64,...,48,65,66,69,6,11,10,8,8,2015
3,505942,2015-03-20 00:00:00,61,43,70,43,50,44,38,63,...,47,62,63,66,5,10,9,7,7,2015
4,505942,2007-02-22 00:00:00,61,43,70,43,50,44,38,63,...,47,62,63,66,5,10,9,7,7,2007


In [30]:
player_attributes = player_attributes.drop('date')

In [31]:
player_2016 = player_attributes.filter(player_attributes.year ==2016)

In [32]:
player_2016.count()

14098

In [33]:
player_2016.select(player_2016.player_api_id).distinct().count()

5586

In [51]:
player_striker_2016 = player_2016.groupBy('player_api_id').agg(
                        {
                            'finishing':'avg',
                            'shot_power':'avg',
                            'acceleration':'avg'
                        }) 

In [52]:
player_striker_2016 = player_striker_2016.withColumnRenamed('avg(finishing)','finishing').\
                    withColumnRenamed('avg(acceleration)','acceleration').\
                    withColumnRenamed('avg(shot_power)','shot_power')

In [53]:
player_striker_2016 = player_striker_2016.withColumn('overall',(player_striker_2016.finishing*1+
                                          player_striker_2016.acceleration*1+
                                          player_striker_2016.shot_power*2)/4)

In [54]:
player_striker_2016 = player_striker_2016.drop('finishing','acceleration','shot_power')

In [55]:
player_striker_2016.orderBy(player_striker_2016['overall'].desc()).show(5)

+-------------+-------+
|player_api_id|overall|
+-------------+-------+
|        20276|  89.25|
|        37412|   89.0|
|        38817|  88.75|
|        32118|  88.25|
|        31921|   87.0|
+-------------+-------+
only showing top 5 rows



In [57]:
striker = player_striker_2016.filter(player_striker_2016.overall > 70).sort(player_striker_2016.overall.desc())

In [58]:
striker.show(5)

+-------------+-------+
|player_api_id|overall|
+-------------+-------+
|        20276|  89.25|
|        37412|   89.0|
|        38817|  88.75|
|        32118|  88.25|
|        31921|   87.0|
+-------------+-------+
only showing top 5 rows



In [63]:
striker_details = players_data.join(striker,players_data.player_api_id == striker.player_api_id)

In [64]:
striker_details.limit(5).toPandas()

,player_api_id,player_name,birthday,height,weight,player_api_id,overall
0,114503,Abiola Dauda,1988-02-03 00:00:00,180.34,165,114503,76.250000
1,166648,Carlos Bacca,1986-09-08 00:00:00,180.34,170,166648,83.208333
2,265363,Chris Loewe,1989-04-16 00:00:00,172.72,139,265363,71.666667
3,41890,Claudio Marchisio,1986-01-19 00:00:00,180.34,157,41890,75.250000
4,195413,Daniele Baselli,1992-03-23 00:00:00,182.88,154,195413,72.666667


In [65]:
striker_details2 = players_data.join(striker,['player_api_id'])

In [69]:
top5 = striker_details2.orderBy(striker_details2['overall'].desc())

In [70]:
top5.limit(5).toPandas()

,player_api_id,player_name,birthday,height,weight,overall
0,20276,Hulk,1986-07-25 00:00:00,180.34,187,89.25
1,37412,Sergio Aguero,1988-06-02 00:00:00,172.72,163,89.00
2,38817,Carlos Tevez,1984-02-05 00:00:00,172.72,157,88.75
3,32118,Lukas Podolski,1985-06-04 00:00:00,182.88,183,88.25
4,31921,Gareth Bale,1989-07-16 00:00:00,182.88,163,87.00


In [71]:
from pyspark.sql.functions import broadcast
striker_details3 = players_data.select('player_api_id','player_name').\
                    join(
                        broadcast(striker),
                        ['player_api_id'],'inner')

In [72]:
striker_details3.orderBy(striker_details2['overall'].desc()).limit(5).toPandas()

,player_api_id,player_name,overall
0,20276,Hulk,89.25
1,37412,Sergio Aguero,89.00
2,38817,Carlos Tevez,88.75
3,32118,Lukas Podolski,88.25
4,31921,Gareth Bale,87.00


In [73]:
player_heading_accuracy = player_attributes.select('player_api_id','heading_accuracy').\
                            join(broadcast(players_data),
                                player_attributes.player_api_id == players_data.player_api_id)

In [74]:
player_heading_accuracy.limit(5).show()

+-------------+----------------+-------------+------------------+-------------------+------+------+
|player_api_id|heading_accuracy|player_api_id|       player_name|           birthday|height|weight|
+-------------+----------------+-------------+------------------+-------------------+------+------+
|       505942|              71|       505942|Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
|       505942|              71|       505942|Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
|       505942|              71|       505942|Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
|       505942|              70|       505942|Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
|       505942|              70|       505942|Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
+-------------+----------------+-------------+------------------+-------------------+------+------+



In [77]:
short_count = spark.sparkContext.accumulator(0)
medium_low_count = spark.sparkContext.accumulator(0)
medium_high_count = spark.sparkContext.accumulator(0)
tall_count = spark.sparkContext.accumulator(0)

In [79]:
def count_player_height_category(row):
    height = float(row.height)
    
    if height <= 175:
        short_count.add(1)
    elif height <= 183 and height > 175:
        medium_low_count.add(1)
    elif height <= 195 and height > 183:
        medium_high_count.add(1)
    elif (height>195):
        tall_count.add(1)

In [80]:
player_heading_accuracy.foreach(lambda x: count_player_height_category(x))

In [81]:
height_category = [short_count.value,
                  medium_low_count.value,
                  medium_high_count.value,
                  tall_count.value]
height_category

[18977, 97399, 61518, 3371]

In [84]:
striker_details2.select('player_api_id','overall').coalesce(1).write.option('header','true').csv('players_overall.csv')

In [85]:
striker_details2.select('player_api_id','overall').write.json('players_overall.json')